# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770889711212                   -0.52    9.0    178ms
  2   -2.772148893775       -2.90       -1.32    1.0    108ms
  3   -2.772170282769       -4.67       -2.43    1.0    110ms
  4   -2.772170654891       -6.43       -3.15    1.0    115ms
  5   -2.772170722916       -7.17       -4.56    2.0    133ms
  6   -2.772170722994      -10.11       -4.70    1.0    127ms
  7   -2.772170723013      -10.72       -5.53    1.0    122ms
  8   -2.772170723014      -11.86       -5.76    1.0    541ms
  9   -2.772170723015      -12.09       -6.32    2.0    132ms
 10   -2.772170723015      -13.41       -6.87    1.0    131ms
 11   -2.772170723015      -14.40       -7.49    2.0    182ms
 12   -2.772170723015   +  -13.75       -8.14    1.0    120ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735579751937118

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770771552221                   -0.52    9.0    203ms
  2   -2.772060016364       -2.89       -1.33    1.0    122ms
  3   -2.772082860822       -4.64       -2.40    1.0    110ms
  4   -2.772083337540       -6.32       -3.11    1.0    116ms
  5   -2.772083417620       -7.10       -4.64    2.0    151ms
  6   -2.772083417806       -9.73       -5.11    2.0    135ms
  7   -2.772083417810      -11.43       -5.72    1.0    557ms
  8   -2.772083417811      -12.21       -6.44    1.0    115ms
  9   -2.772083417811      -13.08       -6.58    2.0    130ms
 10   -2.772083417811      -14.03       -7.35    1.0    139ms
 11   -2.772083417811      -14.18       -8.48    1.0    118ms

Polarizability via ForwardDiff:       1.7725349609282757
Polarizability via finite difference: 1.7735579751937118
